In [1]:
import pandas as pd
import numpy as np
import astropy.utils.data
from astropy.coordinates import SkyCoord,concatenate
import astropy.units as u
import datetime as _datetime
from astropy.io import fits as _fits
from astropy.coordinates import FK5,ICRS
from astropy.io import ascii
from astropy.coordinates import solar_system_ephemeris
from astropy.time import Time

# Join sources from 200219 and 191213 together

In [2]:
df_200219=pd.read_csv('200219_results.csv')
df_191213=pd.read_csv('191213g_results.csv')
final_results=pd.concat([df_200219,df_191213], axis=0)
final_results=final_results.reset_index(drop=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,5,6,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,6,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
final_results_ps1=final_results[final_results['source']=='PS1']
final_results_glade=final_results[final_results['source']=='GLADE']

final_results_ps1[['our_id','RA','Dec']].to_csv('ps1_sources_for_gaia.csv')
final_results_glade[['our_id','RA','Dec']].to_csv('glade_sources_for_gaia.csv')
final_results[['our_id','source','RA','Dec']].to_csv('all_sources_for_gaia.csv')


In [4]:


final_results.rename(columns={'RA':'ra_glade'}, inplace=True)
final_results.rename(columns={'Dec':'dec_glade'}, inplace=True)
#final_results['ra_glade']=final_results['RA']

print(list(final_results.columns))


['Unnamed: 0', 'PGC', 'GWGC', 'HyperLEDA', '2MASS', 'SDSS', 'flag1', 'ra_glade', 'dec_glade', 'dist', 'dist_err', 'z', 'Bmag', 'e_Bmag', 'AbsBMag', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'eKmag', 'flag2', 'flag3', 'source', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'objID', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'htmID', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'cx', 'cy', 'cz', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column1', 'Column2', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', '

## Add in the xmatch between GLADE and PS1

In [5]:
glade_ps1_crossmatch=pd.read_csv('gladecheck1arsec2_Lana_S.csv')

glade_ps1_crossmatch.rename(columns={'RA':'ra_glade'}, inplace=True)
glade_ps1_crossmatch.rename(columns={'DEC':'dec_glade'}, inplace=True)

#glade_ps1_crossmatch['Dec']=glade_ps1_crossmatch['DEC']


In [6]:
print(list(glade_ps1_crossmatch.columns))

['our_id', 'ra_glade', 'dec_glade', 'ObjID', 'Column1', 'Column2', 'cx', 'cy', 'cz', 'htmID', 'distance', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'Column3', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'Column4', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMean', 'decMean', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'Column5', 'Column6', 'Column7', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column8', 'Column9', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagStd', 'gMeanApMagNpt', 'gFlags', 'rQfPerfect', 'rMeanPSFMag', 'rMeanPSFMa

### Only take closest crossmatched source

In [7]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

glade_ps1_crossmatch['separation']=np.abs(sep(glade_ps1_crossmatch['ra_glade'].values,glade_ps1_crossmatch['dec_glade'].values,glade_ps1_crossmatch['raMean'].values,glade_ps1_crossmatch['decMean'].values))
glade_ps1_crossmatch=glade_ps1_crossmatch.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

#glade_ps1_crossmatch['RA']=xmatch_glade_ps1_cut['ra']
#xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']

### Combine with sources table to get PS columns for GLADE xmatched sources

In [8]:
# merge PS1 and glade crossmatch columns
#final_results = final_results.merge(glade_ps1_crossmatch, how = 'outer', on = ['our_id', 'our_id'])

df1 = final_results.set_index(["our_id"])
df2 = glade_ps1_crossmatch.set_index(["our_id"])

final_results = df1.combine_first(df2).reset_index()
#final_results=pd.concat([final_results,glade_ps1_crossmatch])

In [9]:
display(final_results[['ra_glade', 'raMean','Bmag']][final_results['Bmag'].notna()])

,ra_glade,raMean,Bmag
0,86.420349,86.420455,14.120
1,46.854248,46.854101,15.254
2,330.137177,NaN,12.801
3,255.468353,255.468240,14.651
4,101.763191,101.763295,16.366
...,...,...,...
77540,319.002000,319.002081,18.510
77551,319.304000,319.303917,18.520
77562,316.515000,316.514742,18.270
77573,317.043000,317.042984,18.050


# Add Gaia DR3 xmatch

In [10]:
xmatch_ps1_only=pd.read_csv('gaia_ps1_xmatch_result.csv')
xmatch_all=pd.read_csv('all_xmatch_dr3.csv')

xmatch_all.rename(columns={'ra.1':'ra_xmatch'}, inplace=True)
xmatch_all.rename(columns={'dec.1':'dec_xmatch'}, inplace=True)

xmatch_all.rename(columns={'ra':'ra_gaia'}, inplace=True)
xmatch_all.rename(columns={'dec':'dec_gaia'}, inplace=True)


df1 = xmatch_all.set_index(["our_id"])
df2 = glade_ps1_crossmatch.set_index(["our_id"])

xmatch_all = df1.combine_first(df2).reset_index()

In [11]:
print(list(xmatch_all.columns))
print(len(xmatch_all))

['our_id', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'ObjID', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_gof_al', 'astrometric_matched_transits', 'astrometric_n_bad_obs_al', 'astrometric_n_good_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_obs_al', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_sigma5d_max', 'b', 'batchID', 'beta', 'bp_g', 'bp_rp', 'col1', 'cx', 'cy', 'cz', 'decMean', 'decMeanErr', 'decStack', 'decStackErr', 'dec_error', 'dec_gaia', 'dec_glade', 'dec_parallax_corr', 'dec_pmdec_corr', 'dec_pmra_corr', 'dec_pseudocolour_corr', 'dec_xmatch', 'designation', 'dist', 'distance', 'dr2_radial_velocity', 'dr2_radial_velocity_error', 'dr2_rv_nb_transits', 'dr2_rv_template_fe_h', 'dr2_rv_template_logg', 'dr2_rv_template_teff', 'duplicated_source', 'dvoRegionID', 'ecl_lat', 'ecl_lon', 'epochMean', 'gFlags', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMa

# Identify sources not crossmatched

In [12]:
common = final_results.merge(xmatch_all,on=['our_id','our_id'])

not_xmatched=final_results[(~final_results.our_id.isin(common.our_id))&(~final_results.our_id.isin(common.our_id))]
print(len(not_xmatched))

15578


# Make a cut on astrometric error and proper motion

In [13]:
xmatch_all['uwe']=np.sqrt(xmatch_all['astrometric_chi2_al']/(xmatch_all['astrometric_n_good_obs_al']-5))
print(min(xmatch_all['uwe']), max(xmatch_all['uwe']), np.nanmin(xmatch_all['pmra']), np.nanmax(xmatch_all['pmdec']))
#and cut on pmra pmdec


0.7633299518456061 166.74739273060956 -75.2187536114139 69.89171461007075


In [14]:
print(np.mean(xmatch_all['uwe']),np.mean(xmatch_all['pmra']))

2.216509638003318 -2.1902082615394174


In [15]:
xmatch_ps1_only=xmatch_all[xmatch_all['source']=='PS1']
xmatch_GLADE_only=xmatch_all[xmatch_all['source']=='GLADE']

xmatch_cut=xmatch_ps1_only[(xmatch_ps1_only['uwe']>1.5) & (np.abs(xmatch_ps1_only['pmra']<3) | (xmatch_ps1_only['pmra']==np.nan)) & (np.abs(xmatch_ps1_only['pmdec']<3) | (xmatch_ps1_only['pmdec']==np.nan))]
print(len(xmatch_cut))

2640


# Recombine 'good' Gaia sources with GLADE sources

### i.e. keep GLADE sources no matter what the errors

In [16]:
xmatch_glade_ps1_cut=pd.concat([xmatch_cut,xmatch_GLADE_only], axis=0)
print(len(xmatch_glade_ps1_cut))

13766


# If there are duplicate PS1 crossmatches, only take closest one

In [17]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

xmatch_glade_ps1_cut['separation']=np.abs(sep(xmatch_glade_ps1_cut['ra_gaia'].values,xmatch_glade_ps1_cut['dec_gaia'].values,xmatch_glade_ps1_cut['ra_xmatch'].values,xmatch_glade_ps1_cut['dec_xmatch'].values))
xmatch_glade_ps1_cut=xmatch_glade_ps1_cut.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

#xmatch_glade_ps1_cut['RA']=xmatch_glade_ps1_cut['ra']
#xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']
#

# Merge 'good' Gaia xmatched sources with non-xmatched sources

In [18]:
total_cut_df = pd.concat([not_xmatched,xmatch_glade_ps1_cut])

In [19]:
print(list(total_cut_df.columns))
print(len(total_cut_df))

['our_id', '2MASS', 'AbsBMag', 'Bmag', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'GWGC', 'Hmag', 'HyperLEDA', 'Jmag', 'Kmag', 'ObjID', 'PGC', 'SDSS', 'Unnamed: 0', 'b', 'batchID', 'beta', 'cx', 'cy', 'cz', 'decMean', 'decMeanErr', 'decStack', 'decStackErr', 'dec_glade', 'dist', 'dist_err', 'distance', 'dvoRegionID', 'eKmag', 'e_Bmag', 'e_Hmag', 'e_Jmag', 'epochMean', 'flag1', 'flag2', 'flag3', 'gFlags', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagNpt', 'gMeanApMagStd', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagNpt', 'gMeanKronMagStd', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagMax', 'gMeanPSFMagMin', 'gMeanPSFMagNpt', 'gMeanPSFMagStd', 'gQfPerfect', 'htmID', 'iFlags', 'iMeanApMag', 'iMeanApMagErr', 'iMeanApMagNpt', 'iMeanApMagStd', 'iMeanKronMag', 'iMeanKronMagErr', 'iMeanKronMagNpt', 'iMeanKronMagStd', 'iMeanPSFMag', 'iMeanPSFMagErr', 'iMeanPSFMagMax', 'iMeanPSFMagMin', 'iMeanPSFMagNpt', 'iMeanPSFMagStd', 'iQfPerfect', 'ip

# Load the template FITS

In [20]:
template_hdulist = _fits.open('WS2021A1-003_CatalogueTemplate.fits')
template_primary_hdu = template_hdulist[0]
template_hdu = template_hdulist[1]
template_column_names = [col.name for col in template_hdu.columns]
print(template_column_names)


['CNAME', 'TARGSRVY', 'TARGPROG', 'TARGCAT', 'TARGID', 'TARGNAME', 'TARGPRIO', 'TARGUSE', 'TARGCLASS', 'PROGTEMP', 'OBSTEMP', 'GAIA_ID', 'GAIA_DR', 'GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR', 'HEALPIX', 'IFU_SPAXEL', 'IFU_PA', 'IFU_DITHER', 'MAG_G', 'MAG_G_ERR', 'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR', 'GAIA_MAG_G', 'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'APS_WL_MIN', 'APS_WL_MAX', 'APS_Z', 'APS_SIGMA', 'APS_TEMPL_LIB', 'APS_TEMPL_LIB_NORM', 'APS_PPXF_WL_MIN', 'APS_PPXF_WL_MAX', 'APS_PPXF_MOM', 'APS_PPXF_DEG_ADD', 'APS_PPXF_DEG_MULT', 'APS_PPXF_NUM_MC', 'APS_GAND_MODE', 'APS_GAND_ERR', 'APS_GAND_RED1', 'APS_GAND_RED2', 'APS_GAND_EBV', 'APS_LS_MODE', 'APS_LS_RES', 'APS_LS_NUM_MC', 'APS_SSP_NUM_WLKR', 'APS_SSP_NUM_CHAIN', 'APS_IFU_MASK', 'APS_IFU_TSSL_TYPE', 'APS_IFU_TSSL_TARG_SNR', 'APS_IFU_TSSL_MIN_SNR', 'APS_IFU_TSSL_COVAR', 'APS_IFU_SRC_ID', 'APS_IF

## Extension 0 Header

In [21]:
ext0_header=template_hdulist[0].header

#Set the catalogue submitter and CCs

ext0_header['CAT_NME1']='Lana'
ext0_header['CAT_NME2']='Salmon'
ext0_header['CAT_MAIL']='lana.salmon@ucdconnect.ie'
ext0_header['CAT_CC']='p.jonker@sron.nl,morgan.fraser@ucd.ie,lorraine.hanlon@ucd.ie,antonio.martin-carrillo@ucd.ie'
print(repr(ext0_header))



SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT WS2021A1-003 Catalogue Template                                         
DATAMVER= '8.00    '           / WEAVE Data Model Version                       
TRIMESTE= '2021A1  '           / Observing Trimester                            
MAG_G_CM= '' / Survey specific mag column(s) used for MAG_G                     
MAG_R_CM= '' / Survey specific mag column(s) used for MAG_R                     
MAG_I_CM= '' / Survey specific mag column(s) used for MAG_I                     
STL_NME1= 'Peter   '                                                            
STL_NME2= 'Jonker  '                                                            
STL_MAIL= 'p.jonker@sron.nl'

## Extension 1 Header

In [22]:
ext1_header=template_hdulist[1].header
print(repr(ext1_header))

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1136 / length of dimension 1                          
NAXIS2  =                    0 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  113 / number of table fields                         
EXTNAME = 'WS2021A1-003 CATALOGUE' / extension name                             
TTYPE1  = 'CNAME   '           / WEAVE object name from coordinates             
TFORM1  = '20A     '           / data format of field: ASCII Character          
TDISP1  = 'A20     '           / Display format for column                      
TUCD1   = 'meta.id;meta.main

## Calculate Gaia mag errors

In [23]:
# according to dr2 https://www.cosmos.esa.int/documents/29201/1773953/Gaia+DR2+primer+version+1.3.pdf/a4459741-6732-7a98-1406-a1bea243df79?t=1581668739161
total_cut_df['phot_g_mean_mag_err']=1.086/total_cut_df['phot_g_mean_flux_over_error']
total_cut_df['phot_bp_mean_mag_err']=1.086/total_cut_df['phot_bp_mean_flux_over_error']
total_cut_df['phot_rp_mean_mag_err']=1.086/total_cut_df['phot_rp_mean_flux_over_error']

In [24]:
print

<function print>

## Take relevant columns from our catalogue

In [25]:
total_cut_df = total_cut_df.reset_index(drop=True)
FITS_table_from_cat=total_cut_df[['source','our_id','source_id','ra_gaia', 'dec_gaia', 'pmra', 'pmra_error',
                                  'pmdec', 'pmdec_error','parallax', 'parallax_error','phot_g_mean_mag',
                                  'phot_g_mean_mag_err','phot_bp_mean_mag','phot_bp_mean_mag_err',
                                  'phot_rp_mean_mag','phot_rp_mean_mag_err', 'gMeanKronMag', 'gMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'objID', 'raMean', 'decMean',
                                  'gMeanKronMag', 'gMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','yMeanKronMag', 'yMeanKronMagErr','zMeanKronMag', 'zMeanKronMagErr',
                                  'ra_glade', 'dec_glade', 'SDSS', 'Hmag', 'e_Hmag','Jmag', 'e_Jmag',  'Kmag', 'eKmag',
                                  'ra_glade', 'dec_glade', 'SDSS','Bmag', 'e_Bmag'
                                ]]


FITS_table_from_cat.columns = ['COORDCAT','TARGID','GAIA_ID','GAIA_RA', 'GAIA_DEC', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 
                               'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR','GAIA_MAG_G', 
                               'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 
                               'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'MAG_G', 'MAG_G_ERR', 
                               'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR',
                               'PS_ID', 'PS_RA', 'PS_DEC',
                               'PS_MAG_G', 'PS_MAG_G_ERR', 'PS_MAG_I', 'PS_MAG_I_ERR', 
                               'PS_MAG_R', 'PS_MAG_R_ERR', 'PS_MAG_Y', 'PS_MAG_Y_ERR', 'PS_MAG_Z', 'PS_MAG_Z_ERR',
                                'NIRCAT_RA',  'NIRCAT_DEC',  'NIRCAT_ID', 'NIRCAT_MAG_H', 'NIRCAT_MAG_H_ERR', 
                               'NIRCAT_MAG_J', 'NIRCAT_MAG_J_ERR', 'NIRCAT_MAG_K', 'NIRCAT_MAG_K_ERR',
                               'OPTCAT_RA', 'OPTCAT_DEC',   'OPTCAT_ID', 'OPTCAT_MAG_B', 'OPTCAT_MAG_B_ERR'
                              ]

FITS_table_from_cat.reset_index()
#survey identifier
FITS_table_from_cat['TARGSRVY']='WS2021A1-003'

#CNAME is set by WEAVE
FITS_table_from_cat['CNAME']=''

#optional personal name for programme/catalogue
FITS_table_from_cat['TARGPROG']='GW-galaxy'
FITS_table_from_cat['TARGCAT']='WS2021A1-003.fits'

#T for target, GALAXY
FITS_table_from_cat['TARGUSE']='T'
FITS_table_from_cat['TARGCLASS']='GALAXY'

# PROGTEMP for MOS LR 3x1020s 60min OB
FITS_table_from_cat['PROGTEMP']='11331'

#OBSTEMP for max airmass=2.4, moon dist=90 deg, 
#max sky brightness=20.5, max seeing = 1.3'', transparency=0.8
FITS_table_from_cat['OBSTEMP']='GAFAD'

#TARGNAME is the same as unique id
FITS_table_from_cat['TARGNAME']=FITS_table_from_cat['TARGID']

#TARGPRIO = 10.0 for GLADE galaxies
#For the rest, set according to 
FITS_table_from_cat['TARGPRIO']=float(10)

    
def DR (df):
    if df['source'] == 'GLADE' :
        return 'V2'
    else:
        return ''
def glade (df):
    if df['source'] == 'GLADE' :
        return 'GLADE'
    else:
        return ''
    
def gaia (df):
    if df['GAIA_ID']!='':
        return '3'
    else:
        return ''

def PS (df):
    if df['PS_ID']!='':
        return '2'
    else:
        return ''
    
# ID columns are strings
FITS_table_from_cat['GAIA_ID']=FITS_table_from_cat['GAIA_ID'].astype(str)
FITS_table_from_cat['PS_ID']=FITS_table_from_cat['PS_ID'].astype(str)
FITS_table_from_cat['OPTCAT_ID']=FITS_table_from_cat['OPTCAT_ID'].astype(str)
FITS_table_from_cat['NIRCAT_ID']=FITS_table_from_cat['NIRCAT_ID'].astype(str)

# Null values for ID columns should be empty strings
FITS_table_from_cat.loc[FITS_table_from_cat.GAIA_ID == 'nan', ['GAIA_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.NIRCAT_ID == 'nan', ['NIRCAT_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.OPTCAT_ID == 'nan', ['OPTCAT_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.PS_ID == 'nan', ['PS_ID']] = ''

# only give NIRCAT/OPTCAT columns for GLADE galaxies.
FITS_table_from_cat['NIRCAT_DR']=total_cut_df.apply (lambda total_cut_df: DR(total_cut_df), axis=1)
FITS_table_from_cat['OPTCAT_DR']=total_cut_df.apply (lambda total_cut_df: DR(total_cut_df), axis=1)
FITS_table_from_cat['NIRCAT']=total_cut_df.apply (lambda total_cut_df: glade(total_cut_df), axis=1)
FITS_table_from_cat['OPTCAT']=total_cut_df.apply (lambda total_cut_df: glade(total_cut_df), axis=1)

#Only give Gaia/PS columns for sources crossmatched with either.
#FITS_table_from_cat['GAIA_DR']=FITS_table_from_cat.apply (lambda FITS_table_from_cat: gaia(FITS_table_from_cat), axis=1)
FITS_table_from_cat['PS_DR']=FITS_table_from_cat.apply (lambda FITS_table_from_cat: PS(FITS_table_from_cat), axis=1)
FITS_table_from_cat['GAIA_DR']='3'
FITS_table_from_cat['GAIA_EPOCH']=2016.0  
# PS nulls are -999 - set to nan
FITS_table_from_cat=FITS_table_from_cat.replace(-999.000000, np.nan)


# Set APS / IFU columns to the default by setting them to null

FITS_table_from_cat['APS_PPXF_MOM']=-1
FITS_table_from_cat['APS_FLAG'] =''
FITS_table_from_cat['IFU_SPAXEL']=''
FITS_table_from_cat['APS_Z']=np.nan
FITS_table_from_cat['APS_TEMPL_LIB']=''
FITS_table_from_cat['APS_GAND_EBV']=-1
FITS_table_from_cat['APS_GAND_ERR']=-1
FITS_table_from_cat['APS_TEMPL_LIB_NORM']=-1
FITS_table_from_cat['APS_IFU_TSSL_MIN_SNR'] =np.nan
FITS_table_from_cat['IFU_PA'] =np.nan
FITS_table_from_cat['IFU_DITHER'] =float(0)
FITS_table_from_cat['APS_WL_MIN'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_TYPE'] =''
FITS_table_from_cat['GENCAT_INFO'] =''
FITS_table_from_cat['APS_SSP_NUM_WLKR'] =-1
FITS_table_from_cat['APS_LS_NUM_MC'] =-1
FITS_table_from_cat['APS_PPXF_DEG_MULT'] =-1
FITS_table_from_cat['APS_GAND_MODE'] =-1
FITS_table_from_cat['APS_GAND_RED2'] =np.nan
FITS_table_from_cat['APS_PPXF_NUM_MC'] =-1
FITS_table_from_cat['APS_IFU_SRC_DEC']=np.nan
FITS_table_from_cat['APS_IFU_SRC_ID'] =''
FITS_table_from_cat['APS_LS_RES'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_COVAR'] =-1
FITS_table_from_cat['APS_SSP_NUM_CHAIN'] =-1
FITS_table_from_cat['HEALPIX'] =-1
FITS_table_from_cat['APS_LS_MODE'] =-1
FITS_table_from_cat['APS_WL_MAX'] =np.nan
FITS_table_from_cat['APS_IFU_MASK'] =-1
FITS_table_from_cat['APS_PPXF_DEG_ADD'] =-1
FITS_table_from_cat['APS_PPXF_WL_MAX'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_TARG_SNR'] =np.nan
FITS_table_from_cat['APS_GAND_RED1'] =np.nan
FITS_table_from_cat['APS_IFU_SRC_RA'] =np.nan
FITS_table_from_cat['APS_SIGMA'] =np.nan
FITS_table_from_cat['APS_PPXF_WL_MIN']=np.nan

# add spec or phot z from GLADE
def glade_spec_z (df):
    if df['flag2']==1:
        return df['z']
    else:
        return np.nan
    
def glade_phot_z (df):
    if df['flag2']==3:
        return df['z']
    else:
        return np.nan
    
FITS_table_from_cat['PHOTOZCAT_Z']=total_cut_df.apply (lambda total_cut_df: glade_phot_z(total_cut_df), axis=1)
FITS_table_from_cat['SPECZCAT_Z']=total_cut_df.apply (lambda total_cut_df: glade_spec_z(total_cut_df), axis=1)

FITS_table_from_cat['PHOTOZCAT_DR']= np.where(FITS_table_from_cat.PHOTOZCAT_Z.notnull(), 'V2','')
FITS_table_from_cat['SPECZCAT_DR']= np.where(FITS_table_from_cat.SPECZCAT_Z.notnull(), 'V2','')
FITS_table_from_cat['PHOTOZCAT']= np.where(FITS_table_from_cat.PHOTOZCAT_Z.notnull(), 'GLADE','')
FITS_table_from_cat['SPECZCAT']= np.where(FITS_table_from_cat.SPECZCAT_Z.notnull(), 'GLADE','')

FITS_table_from_cat['PHOTOZCAT_ID'] = np.where(FITS_table_from_cat.PHOTOZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_ID,'')
FITS_table_from_cat['SPECZCAT_ID'] = np.where(FITS_table_from_cat.SPECZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_ID,'')
FITS_table_from_cat['PHOTOZCAT_RA'] = np.where(FITS_table_from_cat.PHOTOZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_RA,np.nan)
FITS_table_from_cat['SPECZCAT_RA'] = np.where(FITS_table_from_cat.SPECZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_RA,np.nan)
FITS_table_from_cat['PHOTOZCAT_DEC'] = np.where(FITS_table_from_cat.PHOTOZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_DEC,np.nan)
FITS_table_from_cat['SPECZCAT_DEC'] = np.where(FITS_table_from_cat.SPECZCAT_Z.notnull(), FITS_table_from_cat.OPTCAT_DEC,np.nan)

FITS_table_from_cat['PHOTOZCAT_ID']=FITS_table_from_cat['PHOTOZCAT_ID'].astype(str)
FITS_table_from_cat['SPECZCAT_ID']=FITS_table_from_cat['SPECZCAT_ID'].astype(str)
FITS_table_from_cat.loc[FITS_table_from_cat.PHOTOZCAT_ID == 'nan', ['PHOTOZCAT_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.SPECZCAT_ID == 'nan', ['SPECZCAT_ID']] = ''



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

# Set TARGPRIO based on magnitude

In [26]:
# check the magnitude of sources from PS1 and apply weights. If GLADE, set to 10.0.
mags=FITS_table_from_cat['PS_MAG_R']
minval= min(mags)
maxval= max(mags)
total_cut_df['normalized'] = (10-1)*(1.0-(mags-minval)/(maxval-minval))+1
FITS_table_from_cat['TARGPRIO'] = np.where(FITS_table_from_cat.PS_ID!='', total_cut_df.normalized,float(10))
FITS_table_from_cat.loc[FITS_table_from_cat.OPTCAT == 'GLADE', ['TARGPRIO']] = float(10)

## If we don't have Gaia RA/Dec, transform PS1/GLADE to J2016 and put this into Gaia_RA, Gaia_dec columns

In [27]:
# Take PS or GLADE ra,dec,epoch
for t in range(0,len(FITS_table_from_cat)):
    if FITS_table_from_cat['GAIA_ID'][t]=='':
        if FITS_table_from_cat['PS_ID'][t]!='':
            FITS_table_from_cat['GAIA_RA'][t]=FITS_table_from_cat['PS_RA'][t]
            FITS_table_from_cat['GAIA_DEC'][t]=FITS_table_from_cat['PS_DEC'][t]
            FITS_table_from_cat['GAIA_EPOCH'][t]=2016.0
        else:
            FITS_table_from_cat['GAIA_RA'][t]=FITS_table_from_cat['OPTCAT_RA'][t]
            FITS_table_from_cat['GAIA_DEC'][t]=FITS_table_from_cat['OPTCAT_DEC'][t]
            FITS_table_from_cat['GAIA_EPOCH'][t]=2016.0
         

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by Inte

## Fill in source of Mag G, Mag R, Mag I columns

In [28]:
# give a list of the columns where the Mag_GRI column came from. 
mag_source_g=[]
mag_source_r=[]
mag_source_i=[]

for t in range(0,len(total_cut_df)):
    if total_cut_df['source'][t]=='GLADE':
        mag_source_g.append('PS_MAG_G')
        mag_source_r.append('PS_MAG_R')
        mag_source_i.append('PS_MAG_I')
    else:
        mag_source_g.append('PS_MAG_G')
        mag_source_r.append('PS_MAG_R')
        mag_source_i.append('PS_MAG_I')

ext0_header['MAG_G_CM']='|'.join(mag_source_g)
ext0_header['MAG_R_CM']='|'.join(mag_source_r)
ext0_header['MAG_I_CM']='|'.join(mag_source_i)

# Study new dataframe

In [29]:
pd.set_option('display.max_columns', None)
FITS_table_from_cat=FITS_table_from_cat[template_column_names]

# put a cut of 1.0 on gaia_mag_bp_err as it seems to go above 1?

FITS_table_from_cat['GAIA_MAG_BP_ERR'][FITS_table_from_cat['GAIA_MAG_BP_ERR']>1]=1
display(FITS_table_from_cat)



,CNAME,TARGSRVY,TARGPROG,TARGCAT,TARGID,TARGNAME,TARGPRIO,TARGUSE,TARGCLASS,PROGTEMP,OBSTEMP,GAIA_ID,GAIA_DR,GAIA_RA,GAIA_DEC,GAIA_EPOCH,GAIA_PMRA,GAIA_PMRA_ERR,GAIA_PMDEC,GAIA_PMDEC_ERR,GAIA_PARAL,GAIA_PARAL_ERR,HEALPIX,IFU_SPAXEL,IFU_PA,IFU_DITHER,MAG_G,MAG_G_ERR,MAG_R,MAG_R_ERR,MAG_I,MAG_I_ERR,GAIA_MAG_G,GAIA_MAG_G_ERR,GAIA_MAG_BP,GAIA_MAG_BP_ERR,GAIA_MAG_RP,GAIA_MAG_RP_ERR,APS_WL_MIN,APS_WL_MAX,APS_Z,APS_SIGMA,APS_TEMPL_LIB,APS_TEMPL_LIB_NORM,APS_PPXF_WL_MIN,APS_PPXF_WL_MAX,APS_PPXF_MOM,APS_PPXF_DEG_ADD,APS_PPXF_DEG_MULT,APS_PPXF_NUM_MC,APS_GAND_MODE,APS_GAND_ERR,APS_GAND_RED1,APS_GAND_RED2,APS_GAND_EBV,APS_LS_MODE,APS_LS_RES,APS_LS_NUM_MC,APS_SSP_NUM_WLKR,APS_SSP_NUM_CHAIN,APS_IFU_MASK,APS_IFU_TSSL_TYPE,APS_IFU_TSSL_TARG_SNR,APS_IFU_TSSL_MIN_SNR,APS_IFU_TSSL_COVAR,APS_IFU_SRC_ID,APS_IFU_SRC_RA,APS_IFU_SRC_DEC,APS_FLAG,NIRCAT,NIRCAT_DEC,NIRCAT_DR,NIRCAT_ID,NIRCAT_MAG_H,NIRCAT_MAG_H_ERR,NIRCAT_MAG_J,NIRCAT_MAG_J_ERR,NIRCAT_MAG_K,NIRCAT_MAG_K_ERR,NIRCAT_RA,OPTCAT,OPTCAT_DEC,OPTCAT_DR,OPTCAT_ID,OPTCAT_MAG_B,OPTCAT_MAG_B_ERR,OPTCAT_RA,PHOTOZCAT,PHOTOZCAT_DEC,PHOTOZCAT_DR,PHOTOZCAT_ID,PHOTOZCAT_RA,PHOTOZCAT_Z,PS_DEC,PS_DR,PS_ID,PS_MAG_G,PS_MAG_G_ERR,PS_MAG_I,PS_MAG_I_ERR,PS_MAG_R,PS_MAG_R_ERR,PS_MAG_Y,PS_MAG_Y_ERR,PS_MAG_Z,PS_MAG_Z_ERR,PS_RA,SPECZCAT,SPECZCAT_DEC,SPECZCAT_DR,SPECZCAT_ID,SPECZCAT_RA,SPECZCAT_Z
0,,WS2021A1-003,GW-galaxy,WS2021A1-003.fits,191213_10000,191213_10000,10.0,T,GALAXY,11331,GAFAD,,3,334.175000,-43.041000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,GLADE,-43.041000,V2,,NaN,NaN,NaN,NaN,NaN,NaN,334.175000,GLADE,-43.041000,V2,,19.03,0.5,334.175000,,NaN,,,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN
1,,WS2021A1-003,GW-galaxy,WS2021A1-003.fits,191213_10001,191213_10001,10.0,T,GALAXY,11331,GAFAD,,3,334.227000,-42.799300,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,GLADE,-42.799300,V2,,NaN,NaN,NaN,NaN,NaN,NaN,334.227000,GLADE,-42.799300,V2,,19.38,0.5,334.227000,,NaN,,,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN
2,,WS2021A1-003,GW-galaxy,WS2021A1-003.fits,191213_10002,191213_10002,10.0,T,GALAXY,11331,GAFAD,,3,334.271000,-42.971300,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,GLADE,-42.971300,V2,,NaN,NaN,NaN,NaN,NaN,NaN,334.271000,GLADE,-42.971300,V2,,18.16,0.5,334.271000,,NaN,,,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN
3,,WS2021A1-003,GW-galaxy,WS2021A1-003.fits,191213_10007,191213_10007,10.0,T,GALAXY,11331,GAFAD,,3,328.542000,-39.272580,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,GLADE,-39.272580,V2,,NaN,NaN,NaN,NaN,NaN,NaN,328.542000,GLADE,-39.272580,V2,,NaN,NaN,328.542000,,NaN,,,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN
4,,WS2021A1-003,GW-galaxy,WS2021A1-003.fits,191213_10035,191213_10035,10.0,T,GALAXY,11331,GAFAD,,3,330.939000,-40.531120,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,GLADE,-40.531120,V2,,NaN,NaN,NaN,NaN,NaN,NaN,330.939000,GLADE,-40.531120,V2,,18.46,0.5,330.939000,,NaN,,,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# Populate FITS file with this dict

In [30]:
import datetime as _datetime

from astropy.io import fits as _fits


def populate_fits_table_template(fits_template, data_dict, output_filename,
                                 header, update_datetime=True,
                                 checksum=True, overwrite=False):
    """
    Populate a FITS table template with the provided data.

    Parameters
    ----------
    fits_template : list of str
        A FITS template with a primary HDU and a first extension with a table.
    data_dict : dict
        A dictionary with the data. Its keys should contain the name of the
        columns of the table in the first extension of the FITS template. Its
        values should be array-like with the data to populate the table.
    output_filename : str
        The name of the output file which will be created.
    primary_kwds : dict, optional
        A dictionary with a list of keywords and their corresponding values
        which will be written a in the primary header (updated or added).
    update_datetime : bool, optional
        Update DATETIME keyword in the output file.
    checksum : bool, optional
        Add CHECKSUM and DATASUM keywords in the output file.
    overwrite : bool, optional
        Overwrite the output FITS file.
    """
    
    # Read the FITS template
    
    template_hdulist = _fits.open(fits_template)
    template_primary_hdu = template_hdulist[0]
    template_hdu = template_hdulist[1]
    
    # Check that all the columns are available in the dictionary with the data
    
    template_column_names = [col.name for col in template_hdu.columns]
    
    for col_name in template_column_names:
        assert col_name in data_dict.keys()
    
    # Create list of columns as described in the template populated with the
    # provided data
    
    column_list = []

    for col in template_hdu.columns:
        
        column = _fits.Column(name=col.name, format=col.format,
                             disp=col.disp, unit=col.unit, null=col.null,
                             array=list(data_dict[col.name].values()))
        
        column_list.append(column)
    
    
    # Create a HDU from the column list
    
    coldefs = _fits.ColDefs(column_list)

    hdu = _fits.BinTableHDU.from_columns(coldefs)
    
    # Copy the header from the template
    
    hdu.header = template_hdu.header
    
    # Create the primary extension and populate it with the provided information
    
    primary_hdu = template_primary_hdu
    
    #for kwd in primary_kwds.keys():
     #   primary_hdu.header[kwd] = primary_kwds[kwd]
    primary_hdu = header
    # Update the keyword DATETIME if requested (and it exists)
    
    if (update_datetime is True) and ('DATETIME' in primary_hdu.keys()):
        datetime_str = _datetime.datetime.utcnow().strftime(
                           '%Y-%m-%d %H:%M:%S.%f')
        primary_hdu['DATETIME'] = datetime_str
    primary_hdu=_fits.PrimaryHDU(header=primary_hdu)
    # Create a HDU list and save it to a file
    
    hdulist = _fits.HDUList([primary_hdu, hdu])
    hdulist.writeto(output_filename, checksum=checksum, overwrite=overwrite)
    

In [31]:
data_dicts=FITS_table_from_cat.to_dict()
ext0_header['CAT_NME1']='Lana'
ext0_header['CAT_NME2']='Salmon'
ext0_header['CAT_MAIL']='lana.salmon@ucdconnect.ie'
ext0_header['CAT_CC']='p.jonker@sron.nl,morgan.fraser@ucd.ie,lorraine.hanlon@ucd.ie,antonio.martin-carrillo@ucd.ie'

ext0_header['MAG_G_CM']='PS_MAG_G'
ext0_header['MAG_R_CM']='PS_MAG_R'
ext0_header['MAG_I_CM']='PS_MAG_I'

prim_header=ext0_header
# A dictionary of the catalogue
# A dictionary of the keywords for primary extension
populate_fits_table_template('WS2021A1-003_CatalogueTemplate.fits', data_dicts, 'WS2021A1-003.fits',
                                 prim_header, update_datetime=True,
                                 checksum=True, overwrite=True)